Basic chatbot to act as a clinician to diagnose clinical cases

Load required libraries

see the following URL
https://docs.science.ai.cam.ac.uk/hands-on-llms/2_open_ai/



In [31]:
from openai import OpenAI

In [32]:
import os
import dotenv
from rich import print as rprint

Call chatbot OpenAI API

In [33]:
dotenv.load_dotenv()
# load .env

# get API KEY and store it in variable
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# call OpenAI function
client = OpenAI()


Call API

In [34]:
system_prompt = "You are a clinician and you are diagnosing cases. Given the prompt please give a diagnosis. Please explain all the steps in your reasoning process."
user_query = "A patient has come to you complaining about chest pain. Please give a diagnosis. Please explain all the steps."

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role":"system", "content":system_prompt},
        {"role":"user", "content":user_query}
    ],
    max_tokens=1000 #,
    #temperature=2
)

print(response.choices[0].message.content)


When a patient presents with chest pain, it's crucial to take a systematic approach to reach a diagnosis. Here’s how I would proceed:

### Step 1: Gather a Detailed Patient History
- **Onset**: When did the chest pain start? Was it sudden or gradual?
- **Duration**: How long does the pain last? Is it constant or intermittent?
- **Character**: What is the nature of the pain? Is it sharp, dull, pressure-like, burning, or stabbing?
- **Location**: Where is the pain located? Does it radiate to other parts of the body (e.g., arms, neck, back)?
- **Associated Symptoms**: Are there any other symptoms present such as shortness of breath, nausea, sweating, dizziness, or palpitations?
- **Aggravating/Relieving Factors**: Is there anything that makes the pain worse (e.g., exertion, deep breaths, changing positions) or better (e.g., rest, medications)?
- **Medical History**: Any history of heart disease, chest problems, anxiety, or significant lifestyle factors (smoking, alcohol, stress)?
- **Fami

Slightly improved version of chatbot with more clinical case information

In [35]:
# read in file with case histories
f_str = open("master_prompt.txt", "r")
master_prompt = f_str.read()

system_prompt = "You are a clinician and you are diagnosing cases. Given the prompt please give a diagnosis. Please explain all the steps in your reasoning process."
user_query = "A patient has come to you complaining about chest pain. Please give a diagnosis. Please explain all the steps."
user_query = user_query + master_prompt

#print(user_query)

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role":"system", "content":system_prompt},
        {"role":"user", "content":user_query}
    ],
    max_tokens=1000 #,
    #temperature=2
)

print("*************************************************\n")
print("Output of ClinicalAssistant: \n")
print(response.choices[0].message.content)
print("*************************************************\n")



*************************************************

Output of ClinicalAssistant: 

Based on the detailed clinical case provided, the most likely diagnosis for this patient is **viral pericarditis**.

### Reasoning Process:

1. **Presentation of Symptoms**:
   - The patient presents with acute chest pain lasting for several hours, which is central and radiates to both sides of the chest. This presentation is broad and could indicate various potential causes of chest pain.
   - The pain is relieved by sitting up and leaning forward, a classic sign of pericardial pain, which typically worsens when lying down or with deep inspiration.

2. **History of Previous Symptoms**:
   - The patient has a history of occasional, brief stabbing chest pains over the past two years, which are likely not significant in the context of serious cardiac conditions, as cardiac pain and significant pain typically last longer and have more pronounced features.
   - The recent upper respiratory tract infection wit

Create a chatbot with memory of previous conversation history

In [30]:
print("************************************\n")
print("Intelligent clinical chatbot with memory: \n")

# create a data structure list/dict
system_state = {
    "conversation_history":[]
    }

# create a systemp prompt
system_state["conversation_history"].append({
     "role":"system",
    "content": system_prompt
 })


while True:
    user_input = input("User: ")

    if user_input.lower() in ['exit','done', 'endchat']:
        print("Assistant: Bye! \n")
        break

    #print(user_input)    

    # append user input to system_state
    system_state["conversation_history"].append({ "role":"user", "content":user_input })

    # call OpenAI API
    response_openai = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=system_state["conversation_history"],
        max_tokens=1000 #,
        # temperature = 1.0
    )

    # save the output of the OpenAI chatbot to system_state
    system_state["conversation_history"].append(
        {
            "role":"assistant",
            "content":response_openai.choices[0].message.content
        }
    )  

    # print output of chatbot in this iteration
    print("**************************************************\n")
    print("Output of chatbot ...... \n")
    print(response_openai.choices[0].message.content)
    print("End output \n")
    print("**************************************************\n")


************************************

Intelligent clinical chatbot with memory: 

PLease provide a diagnosis for a patient with chest pain
**************************************************

Output of chatbot ...... 

To provide a diagnosis for a patient presenting with chest pain, I would follow a systematic approach, considering the clinical history, physical examination, and relevant investigations.

### Step 1: Gather Detailed History
1. **Characterization of Pain**:
   - Onset: When did the pain start? Was it sudden or gradual?
   - Quality: Is it sharp, dull, tight, or burning?
   - Duration: How long does it last? Is it intermittent or constant?
   - Location: Where is the pain located? Does it radiate to other areas such as the arms, neck, jaw, or back?
   - Severity: On a scale of 1-10, how severe is the pain?
   - Factors: What were the circumstances surrounding the onset of the pain? Is it associated with physical activity, rest, emotional stress, or meals?

2. **Associated 